In [1]:
#!pip install --upgrade google-api-python-client
#!pip install oauth2client

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import apikey

In [2]:
dev_key = apikey.api_key
api_service_name = "youtube"
api_ver = "v3"

In [3]:
youtube = build(api_service_name, api_ver, developerKey = dev_key)

In [4]:
search_response = youtube.search().list(
    q = "공부하는섭이",
    order = "relevance",
    part = "snippet",
    maxResults = 50
    ).execute()

In [5]:
channel_id = search_response['items'][0]['id']['channelId']

In [6]:
playlists = youtube.playlists().list(
    channelId = channel_id,
    part = "snippet",
    maxResults = 20
    ).execute()

In [7]:
import pandas as pd

ids=[]
titles=[]
for i in playlists['items']:
    ids.append(i['id'])
    titles.append(i['snippet']['title'])
    
df=pd.DataFrame([ids,titles]).T
df.columns=['PlayLists','Titles']

df.head(5)

,PlayLists,Titles
0,PLfkfgvn_kFAU6nfQ-zL_JV8D_SWeHVuRV,Shorts
1,PLfkfgvn_kFAX8jvsuRcoFp6CFPAUehok8,일상 Vlog
2,PLfkfgvn_kFAV4YksRjqGU_H9dWmMRylSG,Q&A / 정보 제공


In [51]:
#Videos' list
dtcu=df['PlayLists'][0]
playlist_videos=youtube.playlistItems().list(
    playlistId=dtcu,
    part='snippet',
    maxResults=50)

playlistitems_list_response=playlist_videos.execute()

video_names=[]
video_ids=[]
date=[]

for v in playlistitems_list_response['items']:
    video_names.append(v['snippet']['title'])
    video_ids.append(v['snippet']['resourceId']['videoId'])
    date.append(v['snippet']['publishedAt'])
    
vdf=pd.DataFrame([date,video_names,video_ids]).T
vdf.columns=['Date','Title','IDS']
#vdf

In [52]:
dtcu2=df['PlayLists'][1]
playlist_videos=youtube.playlistItems().list(
    playlistId=dtcu2,
    part='snippet',
    maxResults=50)

playlistitems_list_response=playlist_videos.execute()

video_names=[]
video_ids=[]
date=[]

for v in playlistitems_list_response['items']:
    video_names.append(v['snippet']['title'])
    video_ids.append(v['snippet']['resourceId']['videoId'])
    date.append(v['snippet']['publishedAt'])
    
vdf2=pd.DataFrame([date,video_names,video_ids]).T
vdf2.columns=['Date','Title','IDS']
#vdf2

In [53]:
dtcu3=df['PlayLists'][2]
playlist_videos=youtube.playlistItems().list(
    playlistId=dtcu3,
    part='snippet',
    maxResults=50)

playlistitems_list_response=playlist_videos.execute()

video_names=[]
video_ids=[]
date=[]

for v in playlistitems_list_response['items']:
    video_names.append(v['snippet']['title'])
    video_ids.append(v['snippet']['resourceId']['videoId'])
    date.append(v['snippet']['publishedAt'])
    
vdf3=pd.DataFrame([date,video_names,video_ids]).T
vdf3.columns=['Date','Title','IDS']
#vdf3

In [55]:
# Appending all lists into one single dataframe and reset the index to aviod index duplication
tdf = vdf.append(vdf2)
tdf = tdf.append(vdf3)
tdf.reset_index(inplace=True, drop=True)
#tdf

In [56]:
# Dislike count value has been removed according to Youtube's policy of not exposing the count of dislike.

import re

category_id=[]
views=[]
likes=[]
comments=[]
mins=[]
seconds=[]
title=[]
date=[]

for u in range(len(tdf)):
    request=youtube.videos().list(
    part='snippet,contentDetails,statistics',
    id=tdf['IDS'][u])
    
    response=request.execute()
    
    if response['items']==[]:
        ids.append('-')
        category_id.append('-')
        views.append('-')
        likes.append('-')
        comments.append('-')
        date.append('-')
        
    else :
        title.append(response['items'][0]['snippet']['title'])
        category_id.append(response['items'][0]['snippet']['categoryId'])
        views.append(response['items'][0]['statistics']['viewCount'])
        likes.append(response['items'][0]['statistics']['likeCount'])
        comments.append(response['items'][0]['statistics']['commentCount'])
        date.append(response['items'][0]['snippet']['publishedAt'])
        
dtcu_df=pd.DataFrame([title,category_id,views,likes,comments,date]).T
dtcu_df.columns=['title','category_id','views','likes','comments','date']
dtcu_df

,title,category_id,views,likes,comments,date
0,한국 음식이 너무 그리운(?) 미국 유학생 #shorts,22,604,15,12,2022-08-10T01:50:16Z
1,미국에서 보내는 첫 생일 #Shorts,22,385,32,14,2022-06-08T13:20:18Z
2,맛이 가버린 텍사스 날씨 #shorts,27,326,14,2,2022-04-05T09:07:09Z
3,새해를 맞이하는 게으른 유학생의 자세 #Shorts,27,269,11,6,2022-01-08T07:46:42Z
4,#Shorts Merry Christmas! 미국 유학생이 크리스마스 즐기는 법,27,317,15,3,2021-12-25T05:49:16Z
5,"IPhone 13 Pro 예구 - 컴퓨터, 아이패드 다음은 아이폰이에요 #shorts",27,431,19,12,2021-09-18T09:00:07Z
6,이것이 미국의 스케일.. This is America #shorts,27,767,18,12,2021-08-11T02:47:58Z
7,낮에 영상 20도인데 밤에 눈이 온다? #shorts,27,179,6,4,2022-02-04T10:00:45Z
8,미국 대학원생 여름학기 일상 Vlog - 벌써 1년?,22,476,51,26,2022-07-31T11:00:31Z
9,미국 인턴 구르는 Vlog - 본격 예비 외노자(되고싶다)의 미국 회사 체험,22,1444,46,38,2022-06-18T23:58:09Z
